<a href="https://colab.research.google.com/github/Pugianf/Big_Data_and_Public_Sector_III-Economic_Development/blob/main/Aula_7_2021_12_16_ModelosBinarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos de Resposta Binária: **Estimando Probabilidades**

Qual o impacto de um ano adicional de estudo na decisão da mulher de entrar (=1) ou não (=0) na força de trabalho? Há diferença na magnitude do impacto entre se obter um ensino fundamental e concluir uma graduação?

E do número de filhos? Há diferença entre filhos pequenos e aqueles com mais idade? E entre o 1º filho e o 4º?



----

## Leitura dos Dados

In [ ]:
## Importando o que for necessário
# Manipulação de dados
import pandas as pd
import numpy as np

# Testes e regressões
from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols, probit, logit  # modelos binários!

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style

style.use('ggplot')  # estilo do R

"""
Vamos usar o statsmodels (probit e logit) para os modelos, mas outro
módulo conhecido para executar regressões logísticas, especialmente no
contexto de Machine Learning, é o sklearn (LogisticRegression)
"""

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


'\nVamos usar o statsmodels (probit e logit) para os modelos, mas outro\nmódulo conhecido para executar regressões logísticas, especialmente no\ncontexto de Machine Learning, é o sklearn (LogisticRegression)\n'

In [ ]:
### Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Lendo os dados
sCaminho = "/content/drive/MyDrive/Projetos/IDP/MBA/SetorPublicoIII/Códigos/Aula 7 - 16_12_2021/"
sArquivo = "mroz.dta"

## Lendo o DataFrame
data = pd.read_stata(f"{sCaminho}{sArquivo}")

# Vendo o tamanho da base
print(f"Linhas: {data.shape[0]}; Colunas: {data.shape[1]}")

In [ ]:
## Vendo o DataFrame
data.head(10)

# inlf: dummy que indica se está ou não na força de trabalho (labor force)
# hours: horas trabalhadas no ano
# kidslt6: número de filhos com menos de 6 anos (less than 6)
# kidsge6: número de filhos com mais de 6 anos (greater than 6)
# wage: salário/hora estimado
# repwage: salário/hora reportado 
# Não usaremos essa diferenciação para o nosso exercício, 
# mas ela é útil para verificar o que se chama de "erros de medida"

# prefixo hus: dados do marido (husband)

# faminc: renda familiar total (family income)
# mtr: taxa marginal do imposto de renda (marginal tax rate)
# motheduc, fatheduc: anos de escolaridade dos pais
# unem: taxa de desemprego na cidade onde mora
# city: dummy que indica se mora em grandes cidades
# exper: anos de experiênca
# nwifeinc: (faminc - wage*hours) (renda familiar não proveniente da mulher)

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,huseduc,huswage,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1.0,1610.0,1.0,0.0,32.0,12.0,3.3540,2.65,2708.0,34.0,12.0,4.0288,16310.0,0.7215,12.0,7.0,5.0,0.0,14.0,10.910060,1.210154,196.0
1,1.0,1656.0,0.0,2.0,30.0,12.0,1.3889,2.65,2310.0,30.0,9.0,8.4416,21800.0,0.6615,7.0,7.0,11.0,1.0,5.0,19.499981,0.328512,25.0
2,1.0,1980.0,1.0,3.0,35.0,12.0,4.5455,4.04,3072.0,40.0,12.0,3.5807,21040.0,0.6915,12.0,7.0,5.0,0.0,15.0,12.039910,1.514138,225.0
3,1.0,456.0,0.0,3.0,34.0,12.0,1.0965,3.25,1920.0,53.0,10.0,3.5417,7300.0,0.7815,7.0,7.0,5.0,0.0,6.0,6.799996,0.092123,36.0
4,1.0,1568.0,1.0,2.0,31.0,14.0,4.5918,3.60,2000.0,32.0,12.0,10.0000,27300.0,0.6215,12.0,14.0,9.5,1.0,7.0,20.100060,1.524272,49.0
5,1.0,2032.0,0.0,0.0,54.0,12.0,4.7421,4.70,1040.0,57.0,11.0,6.7106,19495.0,0.6915,14.0,7.0,7.5,1.0,33.0,9.859054,1.556480,1089.0
6,1.0,1440.0,0.0,2.0,37.0,16.0,8.3333,5.95,2670.0,37.0,12.0,3.4277,21152.0,0.6915,14.0,7.0,5.0,0.0,11.0,9.152048,2.120260,121.0
7,1.0,1020.0,0.0,0.0,54.0,12.0,7.8431,9.98,4120.0,53.0,8.0,2.5485,18900.0,0.6915,3.0,3.0,5.0,0.0,35.0,10.900040,2.059634,1225.0
8,1.0,1458.0,0.0,2.0,48.0,12.0,2.1262,0.00,1995.0,52.0,4.0,4.2206,20405.0,0.7515,7.0,7.0,3.0,0.0,24.0,17.305000,0.754336,576.0
9,1.0,1600.0,0.0,2.0,39.0,12.0,4.6875,4.15,2100.0,43.0,12.0,5.7143,20425.0,0.6915,7.0,7.0,5.0,0.0,21.0,12.925000,1.544899,441.0


In [ ]:
## Descrevendo o DataFrame
data.describe()

## Modelo de Probabilidade Linear (MPL): MQO aplicado a variáveis binárias

O modelo mais simples para estimar a probabilidade consiste em usar MQO para determinar o impacto das variáveis na "escolha" das mulheres de trabalhar ou não.
Para isso, basta colocar a variável binária de decisão como a variável dependente no modelo. 

Contudo, há um *trade-off* entre simplicidade e robustez:

- O MPL pode gerar resultados menores que 0 ou maiores que 1 (por definição, $0 ≤ Prob ≤ 1$). De modo geral, o MPL funciona melhor com os valores de variáveis independentes próximos à **média** da amostra.

- Estatisticamente, a distribuição de uma variável binária é do tipo Bernoulli, e não normal, o que compromete inferências estatísticas em amostras pequenas.

- Ainda falando de estatística, a variânca de uma variável aleatória com distribuição Bernoulli é $var(y | X) = p(1-p)$, onde $p$ é a probabilidade. Como $p$ é função do vetor $X$, o erro jamais será homoscedástico (variância constante ao longo de $X$), o que inviabiliza nossos erros-padrão e torna inválida qualquer inferência estatística não-robusta.

- Por fim, como o modelo é linear, todos os efeitos das variáveis independentes são constantes. No exemplo, passar de 0 pra 1 filho com menos de 6 anos teria o mesmo efeito de passar de 2 pra 3, o que parece pouco verossímil (espera-se que o impacto do primeiro filho seja maior).

In [ ]:
#### Criando o MPL
### Formula (com experiência quadrática, ou seja, retornos côncavos)
formula = "inlf ~ nwifeinc + educ + exper + I(exper**2) + age + kidslt6 + kidsge6"
# Espera-se que:
  # quanto maior a renda não-proveniente da mulher, menor a necessidade dela trabalhar para ajudar nos gastos da casa
  # quanto maior a educação e a experiência, maior a chance de trabalhar (tem mais capital humano e pode ganhar mais)
  # quanto maior o número de filhos, menor a chance de trabalhar (especialmente se forem pequenos)

### Modelo com covariância robusta (MPL sempre é heteroscedástico)
modelo_mpl = ols(formula=formula, data=data).fit(cov_type="HC1", use_t=True)

### Resultados
print(modelo_mpl.summary())

# nwifeinc: quanto maior a renda do resto da família, menor a chance de a mulher trabalhar
# educ: um ano a mais de estudo aumenta em 3,8% a chance da mulher participar do mercado de trabalho, tudo o mais constante
# kidslt6: ter filhos menores diminui muito (-26,18%) a chance de a mulher trabalhar; 
# filhos maiores, por outro lado, não afetam de forma estatisticamente significativa essa probabilidade

In [ ]:
## Vamos realizar um teste de Breusch-Pagan para verificar a heteoscedasticidade
# Lembrando: o teste é um diagnóstico que indica se precisamos ou não usar os erros robustos acima
# e seu resultado não é afetado pela matriz de covariância usada no modelo
teste_heteroscedasticidade = sms.het_breuschpagan(modelo_mpl.resid, modelo_mpl.model.exog)

# Pegando o p-valor
print(f"P-valor: {teste_heteroscedasticidade[3]}")
print("H0: homoscedasticidade.")

# Como esperado, os resíduos são heteroscedásticos (p < 0.05)

NameError: ignored

In [ ]:
## O que é mais interessante é ver os valores previstos pelo modelo
# Pegando os valores previstos e adicionando como coluna
data["pred_mpl"] = modelo_mpl.predict()

data[["inlf", "pred_mpl"]].describe()

# Valor mínimo de -0.34 e máximo de 1.12, o que não faz sentido!

In [ ]:
### Plottando
## Objeto gráfico
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)

## Valores observados (anos de educação é a variável do eixo x)
sns.scatterplot(x="educ", y="inlf", data=data, label="Observados")

## Previstos
sns.scatterplot(x="educ", y="pred_mpl", data=data, label="Previstos")

## Linhas horizontais no 0 e no 1 (limites da probabilidade)
# Valores mínimos e máximos de educação
nMinEduc = data["educ"].min()
nMaxEduc = data["educ"].max()

# Linhas pontilhadas
plt.hlines(y=0, xmin=nMinEduc, xmax=nMaxEduc, linestyles="--", color='k', alpha=0.5)
plt.hlines(y=1, xmin=nMinEduc, xmax=nMaxEduc, linestyles="--", color='k', alpha=0.5)

## Áreas sombreadas para indicar probabilidades inválidas
# Valores mínimos e máximos da probabilidade prevista
nProbMin = data["pred_mpl"].min()
nProbMax = data["pred_mpl"].max()

# Áreas sombreadas
# np.arange: construa um intervalo entre nMinEduc e nMaxEduc com "passos" de 0.1
plt.fill_between(x=np.arange(nMinEduc, nMaxEduc + 1, 0.1), 
                 y1=nProbMin, y2=0, 
                 color='k', alpha=0.3)
plt.fill_between(x=np.arange(nMinEduc, nMaxEduc + 1, 0.1), 
                 y1=1, y2=nProbMax, 
                 color='k', alpha=0.3)

## Títulos e Eixos
plt.title("Previsão do MPL")
plt.xlabel("Anos de Educação")
plt.ylabel("Probabilidade de Estar na Força de Trabalho")

## Legenda
plt.legend(facecolor=None, frameon=False, loc='lower right')

## Veja como temos pontos azuis nas áreas sombreadas, o que não é pra acontecer!

## Probit e Logit

Para contornar os problemas do MPL, podemos usar modelos que se baseiam em funções que, por construção, sempre produzem valores entre 0 e 1.

Esse é o caso da **função de distribuição normal acumulada** - que dará origem ao modelo *probit* - e da **função logística**, que produzirá o *logit* ou a famosa *Regressão Logística*.

Note que as funções usadas não são mais lineares, ou seja, não podemos usar MQO comum para a estimação. Matematicamente, os estimadores usados são os de máxima verossimilhança (MV), que já contém propriedades desejáveis contra heteroscedasticidades e inconsistências/viéses.

### Comportamento das Funções

In [ ]:
### Vamos simular dados para ver os gráficos das funções usadas
## Usando o scipy.stats para gerar a distribuição acumulada de uma função logística
## Para compatibilizar com a norrmal, vamos usar o intervalo de -4 a 4
# cdf: cumulative distribution function
vLogistica = stats.logistic.cdf(np.arange(-4, 4, 0.1))

## Usando o mesmo método para gerar a distribuição acumulada da normal
vNormal = stats.norm.cdf(np.arange(-4, 4, 0.1))

In [ ]:
## Plottando
# Logística
plt.plot(vLogistica)

# Normal
plt.plot(vNormal)

# Legenda
plt.legend(frameon=False, facecolor=None, labels=["Logística", 'Normal'])

# Títulos
plt.title("Logística vs Normal")
plt.ylabel("Probabilidade")

# Logística: mais suave
# Normal: mais concentrada nos valores médios/medianos

### Modelos

#### Logit

In [ ]:
## Usaremos a mesma fórmula do MPL
## Convenientemente, por usarmos o statsmodels, a sintaxe é igual à de uma ols,
## mudando apenas a função
modelo_logit = logit(formula, data).fit(use_t=True, cov_type='HC1')

## Printando sumário
print(modelo_logit.summary())

Note que o sumário é um pouco diferente das regressões comuns, haja vista a não-linearidade e a estimação por máxima-verossimilhança (method=MLE, do inglês *Maximum Likelihood Estimador*). A grosso modo, a comparação com OLS é a seguinte:

- $R^2$ se torna o **Pseudo-R2** e tem interpretação igual a antes (quanto mais próximo de 1, melhor)
- Log-Likelihood e LLR p-value são os valores das estatísticas $F$ e $Prob(F)$, que indicam a signficância do modelo. No caso acima, o p-valor é muito baixo e, logo, o modelo é significante.

Contudo, como o modelo não é linear, os valores dos parâmetros acima não refletem os valores marginais das variáveis (matematicamente, eles não são iguais à derivada parcial em virtude da regra da cadeia). Não podemos dizer, portanto, que 1 ano a mais de educação aumenta em 22% a probabilidade de uma mulher participar da força de trabalho.

De fato, temos que colocar os coeficientes estimados e os valores das características individuais na função do modelo (logística ou normal acumulada).

Há diversas formas de se realizar essa tarefa, sendo os dois mais comuns:

- O Efeito Parcial **na Média** (em inglês, *PEA* ou *mean*). Nesse método, se usa a média de cada coluna (pega-se a média de educação da amostra, média de idade...) e se coloca na função do modelo com os coeficientes estimados acima.

- O Efeito Parcial **Médio** (em inglês, *APE* ou *overall*). Resumidamente, calcula-se o efeito parcial usando as características de cada indivíduo na amostra, e depois se tira a média desses efeitos.

No capítulo 17 de seu livro "Introdução à Econometria" (2011), Wooldridge recomenda usar o *APE*, já que, por ser calculado para cada indivíduo, produz resultados mais precisos.
Contudo, em amostras muito grandes (na ordem de milhões) isso pode ser computacionalmente muito demandante, o que justifica o uso do *PEA* nesses casos.

In [ ]:
## Coletando os efeitos marginais usando o APE (at='overall')
# Vamos especificar count=True para mostrar ao statsmodels que nossas variáveis são discretas
efeitos_marginais_logit = modelo_logit.get_margeff(at='overall', count=True)
print(efeitos_marginais_logit.summary())

O modelo logit indica que o efeito parcial médio de um ano a mais de estudo é de cerca de 3,94%, maior que o observado no MPL. O resultado de kidslt6 é um pouco menor que o observado visto no MPL.

In [ ]:
## Contudo, como o modelo não é linear, os efeitos das variáveis variam dependendo de seu nível
## Podemos, assim, também ver os efeitos no zero, ou seja, uma mudança de 0 para 1, o que não era possível no MPL
# Quando usamos at='zero', não podemos especificar count=True
efeitos_marginais_logit0 = modelo_logit.get_margeff(at='zero')
print(efeitos_marginais_logit0.summary())

# Vemos que os valores de educ aumentam, ou seja, anos adicionais em mulheres com maior escolaridade
# afetam menos sua decisão do que entre as mulheres menos educadas

# Além disso, o efeito do primeiro filho menor de 6 anos é muito mais pronunciado, em linha com o que esperávamos

In [ ]:
## Podemos ver também os resultados usando o PEA
print(modelo_logit.get_margeff(at='mean', count=True).summary())

In [ ]:
## Por fim, grafando os resultados
# Pegando os valores previstos e adicionando como coluna
data["pred_logit"] = modelo_logit.predict()

data[["inlf", "pred_logit"]].describe()

# Nenhum valor menor que 0 ou maior que 1!

In [ ]:
## Plottando
# Objeto gráfico
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)

# Valores observados
sns.scatterplot(x="educ", y="inlf", data=data, label="Observados")

# Previstos
sns.scatterplot(x="educ", y="pred_logit", data=data, label="Previstos Logit")

# Linhas horizontais no 0 e no 1
plt.hlines(y=0, xmin=nMinEduc, xmax=nMaxEduc, linestyles="--", color='k', alpha=0.5)
plt.hlines(y=1, xmin=nMinEduc, xmax=nMaxEduc, linestyles="--", color='k', alpha=0.5)

# Áreas sombreadas para indicar probabilidades inválidas
plt.fill_between(x=np.arange(nMinEduc, nMaxEduc + 1, 0.1), 
                 y1=nProbMin, y2=0, 
                 color='k', alpha=0.3)
plt.fill_between(x=np.arange(nMinEduc, nMaxEduc + 1, 0.1), 
                 y1=1, y2=nProbMax, 
                 color='k', alpha=0.3)

# Títulos e Eixos
plt.title("Previsão do Logit")
plt.xlabel("Anos de Educação")
plt.ylabel("Probabilidade de Estar na Força de Trabalho")

# Legenda
plt.legend(facecolor=None, frameon=False, loc='lower right')

# Nenhum valor na zona sombreada!

#### Probit

In [ ]:
## Seguiremos o mesmo procedimento acima a fim de comparar as estimativas
modelo_probit = probit(formula, data).fit(use_t=True, cov_type='HC1')

## Printando sumário
print(modelo_probit.summary())

# O pseudo R2 do probit é ligeiramente maior (+ 0.0009), mas nada muito grande
# Ambos os modelos são significativos (vide LLR p-value)

In [ ]:
## Captando os efeitos marginais
efeitos_marginais_probit = modelo_probit.get_margeff(at='overall', count=True)
print("Probit:")
print(efeitos_marginais_probit.summary())

print("\nLogit:")
print(efeitos_marginais_logit.summary())

# Educ iguais, mas probit preve um efeito maior para a queda na participação em virtude de filhos pequenos

In [ ]:
## Captando os efeitos marginais no 0
efeitos_marginais_probit0 = modelo_probit.get_margeff(at='zero')
print("Probit:")
print(efeitos_marginais_probit0.summary())

print("\nLogit:")
print(efeitos_marginais_logit0.summary())

# Logit capta um efeito positivo da educação maior quando se analisa os impactos no 0
# Diferentemente do observado nas médias, o efeito de kidslt6 no logit é 1 p.p. maior (em módulo) que o do probit

In [ ]:
## Por fim, grafando os resultados
# Pegando os valores previstos e adicionando como coluna
data["pred_probit"] = modelo_probit.predict()

# Vendo estatísticas descritivas de todas as previsões
data[["inlf", "pred_mpl", "pred_logit", "pred_probit" ]].describe()

In [ ]:
## Plottando
# Objeto gráfico
fig, ax = plt.subplots(figsize=(6, 6), dpi=100)

# Valores observados
sns.scatterplot(x="educ", y="inlf", data=data, label="Observados")

# Previstos
sns.scatterplot(x="educ", y="pred_probit", data=data, label="Previstos Logit")

# Linhas horizontais no 0 e no 1
plt.hlines(y=0, xmin=nMinEduc, xmax=nMaxEduc, linestyles="--", color='k', alpha=0.5)
plt.hlines(y=1, xmin=nMinEduc, xmax=nMaxEduc, linestyles="--", color='k', alpha=0.5)

# Áreas sombreadas para indicar probabilidades inválidas
plt.fill_between(x=np.arange(nMinEduc, nMaxEduc + 1, 0.1), 
                 y1=nProbMin, y2=0, 
                 color='k', alpha=0.3)
plt.fill_between(x=np.arange(nMinEduc, nMaxEduc + 1, 0.1), 
                 y1=1, y2=nProbMax, 
                 color='k', alpha=0.3)

# Títulos e Eixos
plt.title("Previsão do Probit")
plt.xlabel("Anos de Educação")
plt.ylabel("Probabilidade de Estar na Força de Trabalho")

# Legenda
plt.legend(facecolor=None, frameon=False, loc='lower right')

# Novamente, nenhum valor na área sombreada :)

## Exercício

Um *fun fact* sobre os modelos binários é que há uma regra de bolso (que nem sempre funciona) para captar os efeitos marginais das variáveis na média (PEA): no *Probit*, basta dividir os coeficientes do sumário por 2,5, enquanto no *Logit* basta dividir por 4.

Será que é verdade?

In [ ]:
## Vendo os efeitos marginais do modelo probit na média das características
probit_pea = modelo_probit.get_margeff(at="mean", count=True)

print(probit_pea.summary())

In [ ]:
## Criando um DataFrame com os resultados dy/dx acima e com a regra de bolsa
dfComparacaoProbit = pd.DataFrame(
    [probit_pea.margeff, 
     np.array(modelo_probit.params)[1:] / 2.5],
    index=["Efeitos Marginais", "Regra de Bolso"],
    columns=modelo_probit.model.exog_names[1:]
).T

# Coluna de erro
dfComparacaoProbit["Erro"] = dfComparacaoProbit["Efeitos Marginais"] - dfComparacaoProbit["Regra de Bolso"]

# Resultado
dfComparacaoProbit

In [ ]:
## Fazendo a mesma coisa para o logit, só que agora dividindo por 4
# Vendo os efeitos marginais do modelo probit na média das características
logit_pea = modelo_logit.get_margeff(at="mean", count=True)

## Criando um DataFrame com os resultados dy/dx acima e com a regra de bolsa
dfComparacaoLogit = pd.DataFrame(
    [logit_pea.margeff, 
     np.array(modelo_logit.params)[1:] / 4],
    index=["Efeitos Marginais", "Regra de Bolso"],
    columns=modelo_logit.model.exog_names[1:]
).T

# Coluna de erro
dfComparacaoLogit["Erro"] = dfComparacaoLogit["Efeitos Marginais"] - dfComparacaoLogit["Regra de Bolso"]

# Resultado
dfComparacaoLogit

# Obrigado, boas férias e boas festas!